# 프로젝트 - Movielens 영화 추천 실습 (추천 시스템 프로젝트)

# 후기

아직까지는 오디오가 최강의 난이도다.   
진짜 오디오는 아직도 해결을 못한 **극악의 노드**....    

추천 시스템은 돌아가긴 하지만, dataset을 조금 잘못 설정한 것 같다.   
하나하나 movie를 search하면서 봐야하니 귀찮다.   
나중에 merge로 합친 후에 텍스트를 출력하도록 만들어봐야겠다.   
추천시스템 자체는 돌아가지만, 추천의 성능은 매우 구리다.   
넷플릭스와 왓챠 AI 분들께 존경과 박수를...   

# 진행순서

1) 데이터 전처리와 준비   
2) 분석   
3) CSR matrix   
4) AlternatingLeastSquares 모델 구성 및 훈련   
5) 모델 성능 확인 (선호도 파악 및 유사 영화 열거)   
6) 추천 받기   

### 1) 데이터 전처리와 준비

In [1]:
import pandas as pd
import numpy as np
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat' # 데이터 패스
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python') # python으로 읽는다. 컬럼명은 names 인자로 준다.
orginal_data_size = len(ratings)
display(ratings.head()) # 데이터 형태를 확인합시다.
ratings.shape # 데이터 행과 열

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


(1000209, 4)

In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3] # ratings 컬럼에서 3이상만!! -->  ratings 데이터프레임으로!!
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}') # 원래는 1000209개, 3점 이상은 836478개
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}') # 3점 이상은 83.63% 입니다!!

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다. 
# 왜 바꾸냐고? 평점이 높은 만큼 더 많이 봤다라고 생각하기 위해서!!
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
display(movies.head())

movies['title'][1]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


'Jumanji (1995)'

### 2) 분석해봅시다.
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [6]:
# ratings에 있는 유니크한 영화 개수
print("유니크한 영화 개수 :",ratings['movie_id'].nunique())

# ratings에 있는 유니크한 사용자 수
print("유니크한 사용자 수 :",ratings['user_id'].nunique())

# 가장 인기 있는 영화 30개 -> ratings에서 가장 인기 많은 영화를 찾은 뒤, movies에서 상위 30개 제목을 가져온다.
popular_movies_count = ratings.groupby('movie_id')['user_id'].count()
movies_top30 = popular_movies_count.sort_values(ascending=False).head(30)
movies_top30 = movies_top30.keys()

most_popular_movies_list = []
for x in movies_top30 :
    most_popular_movies_list.append(movies['title'][x])
    
print("가장 인기있는 영화 리스트 :",most_popular_movies_list)

# 혹시 모르니 딕셔너리
movie_dict = {}
for x in range(len(movies_top30)):
    movie_dict[movies_top30[x]] = most_popular_movies_list[x]
# print(movie_dict)

유니크한 영화 개수 : 3628
유니크한 사용자 수 : 6039
가장 인기있는 영화 리스트 : ['Brief Encounter (1946)', 'Ladybird Ladybird (1994)', 'Alien (1979)', 'Raging Bull (1980)', 'Something Wicked This Way Comes (1983)', 'Silence of the Lambs, The (1991)', 'Pretty Woman (1990)', 'Big Blue, The (Le Grand Bleu) (1988)', 'Some Kind of Wonderful (1987)', 'Superman (1978)', 'Lassie (1994)', 'Dog of Flanders, A (1999)', 'Pallbearer, The (1996)', 'Rumble in the Bronx (1995)', 'Wishmaster (1997)', 'Secret Garden, The (1993)', 'Army of Darkness (1993)', 'Deadly Friend (1986)', 'Stripes (1981)', 'Strawberry and Chocolate (Fresa y chocolate) (1993)', 'Kansas City (1996)', 'Heathers (1989)', 'Associate, The (1996)', 'Tora! Tora! Tora! (1970)', 'Tales of Terror (1962)', 'Priest (1994)', 'I Love Trouble (1994)', 'M (1931)', 'Jumanji (1995)', 'Go Fish (1994)']


### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [7]:
# my_favorite_movies = ['A Clockwork Orange (1971)', 'Plein Soleil (1960)', 'Leon (1994)', 'The Darkknight (2008)', 'Up (2009)']
# for x in my_favorite_movies :
#     if x in movies['title'] :
#         print(x)
#     else :
#         print("We don't have",x, "in movie list.")
# 내가 좋아하는 영화는 없다.


# 있는 영화 중에 고릅시다.
my_favorite_movies = ['Alien (1979)', 'Raging Bull (1980)', 'Pretty Woman (1990)', 'Silence of the Lambs, The (1991)', 'Jumanji (1995)']

print("내가 좋아하는 영화 5개는 ",my_favorite_movies,"입니당")

# 애네 아이디가 뭐더라
my_favorite_movies_id = []
for idx, title in movie_dict.items(): 
    if title in my_favorite_movies:
        my_favorite_movies_id.append(idx)

print("내가 좋아하는 영화 5개 아이디는 ",my_favorite_movies_id,"입니당")


    
# timestamp는 뭔지 모르겠다 그냥 하자    
# 한번 rating에 추가해봅시다.
my_movie_list = pd.DataFrame({'user_id' : [6041]*5, 'movie_id':my_favorite_movies_id, 'count':[5]*5, 'timestamp':[978298730]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie_list)
    


# 내가 잘 추가 되었는지 확인해봅시다.
# 유니크한 사용자 수가 이전에는 6040명이었음 / 
print("유니크한 사용자 수 :",ratings['user_id'].nunique()) # 추가 되었다.

ratings.tail()

내가 좋아하는 영화 5개는  ['Alien (1979)', 'Raging Bull (1980)', 'Pretty Woman (1990)', 'Silence of the Lambs, The (1991)', 'Jumanji (1995)'] 입니당
내가 좋아하는 영화 5개 아이디는  [1196, 1210, 589, 593, 1] 입니당
유니크한 사용자 수 : 6040


,user_id,movie_id,count,timestamp
0,6041,1196,5,978298730
1,6041,1210,5,978298730
2,6041,589,5,978298730
3,6041,593,5,978298730
4,6041,1,5,978298730


In [8]:
# 전처리를 해야 하나요? --> 밑에서 자꾸 오류나니까 한 번 해봅시다.
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print("내 아이디 인덱스 :",user_to_idx[6041])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 합니다. 
print("쥬만지 인덱스 :",movie_to_idx[1])

내 아이디 인덱스 : 6039
쥬만지 인덱스 : 40


In [23]:
# 해당 코드를 돌리면 영화 아이디 순서가 뒤죽박죽이 된다!! 돌리지 말것!!


# # user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# # 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
# temp_user_ratings = ratings['user_id'].map(user_to_idx.get).dropna()
# if len(temp_user_ratings) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
#     print('user_id column indexing OK!!')
#     ratings['user_id'] = temp_user_ratings   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
# else:
#     print('user_id column indexing Fail!!')

# # artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
# temp_movie_ratings = ratings['movie_id'].map(movie_to_idx.get).dropna()
# if len(temp_movie_ratings) == len(ratings):
#     print('moviet column indexing OK!!')
#     ratings['movie_id'] = temp_movie_ratings
# else:
#     print('movie column indexing Fail!!')

ratings.tail()

,user_id,movie_id,count,timestamp
0,6041,1196,5,978298730
1,6041,1210,5,978298730
2,6041,589,5,978298730
3,6041,593,5,978298730
4,6041,1,5,978298730


### 4) CSR matrix를 직접 만들어 봅시다.

In [10]:
ratings.info()

# CSR matrix 갑니다.
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

print(num_user)
print(num_movie)

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    836483 non-null  int64
 1   movie_id   836483 non-null  int64
 2   count      836483 non-null  int64
 3   timestamp  836483 non-null  int64
dtypes: int64(4)
memory usage: 31.9 MB
6040
3628


<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [11]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)



In [12]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [13]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [38]:
# 첫 빠따는 쥬만지가 맞자.

sungjun, Jumanji = user_to_idx[6039], movie_to_idx[1]
sungjun_vector, Jumanji_vector = als_model.user_factors[6039], als_model.item_factors[1]

In [39]:
sungjun_vector

array([ 2.5495994e+00, -8.6224788e-01, -1.8791540e-01,  7.9482198e-01,
        5.5662923e+00, -4.9543223e-01,  2.1576534e-01, -1.6990745e+00,
        2.6098324e-02,  9.4485527e-01, -5.9766686e-01,  2.2555578e+00,
       -9.1023976e-01, -3.8946372e-01, -3.0188015e-01, -8.3228499e-01,
       -1.3410946e+00,  1.1048502e+00, -2.5588295e+00, -1.7260292e-01,
        4.6380201e-01, -7.4189103e-01, -1.1903168e+00, -2.3375202e-03,
       -6.4618081e-01, -2.6441116e+00,  2.5400394e-01,  2.0994308e+00,
       -1.2252605e+00, -3.6599725e-01,  4.5587519e-01, -9.3207222e-01,
        4.5611572e+00, -3.1141931e-01, -4.5816520e-01, -4.5247903e-01,
       -2.8477341e-01, -8.5362262e-01, -2.1571998e-02,  8.9196366e-01,
       -4.9735017e-02, -2.7338853e+00,  1.8755355e+00, -1.2122713e+00,
        1.4764248e-01,  2.5329709e-01, -2.6853859e+00, -1.7978585e-01,
        3.2318761e+00, -1.0718497e+00,  7.0740975e-02,  4.8718926e-01,
        1.6195968e-01, -3.0352762e+00, -1.8738453e+00, -1.3348523e+00,
      

In [40]:
Jumanji_vector

array([ 0.02480057, -0.00506632, -0.02752275,  0.00683821,  0.00548419,
        0.0006699 , -0.01214437,  0.02091811,  0.03666697,  0.02612253,
       -0.01596926, -0.00918277,  0.0014776 ,  0.02646539, -0.02338996,
       -0.01764669,  0.02524528,  0.0094071 , -0.0380257 , -0.0336464 ,
        0.01196412,  0.0429974 , -0.01598025, -0.01611785,  0.00654157,
       -0.0269997 , -0.00455737,  0.02506502, -0.02914086,  0.02789844,
        0.01764943, -0.01244365,  0.04122737,  0.02010476,  0.00313672,
        0.00072292,  0.00924885, -0.00939887, -0.0306387 ,  0.0375472 ,
        0.02688732,  0.01440808,  0.00536236,  0.00851207, -0.0013808 ,
       -0.00330038,  0.02024124,  0.00205612,  0.02213772,  0.0173508 ,
        0.00727039,  0.04486958,  0.01187072, -0.0212801 ,  0.01709189,
        0.00286341,  0.01087059,  0.05189516,  0.0079086 , -0.00072528,
        0.03044181, -0.03394064, -0.00708742,  0.01288798,  0.01642933,
        0.00646783, -0.03158372,  0.04809976,  0.00843745, -0.01

In [41]:
# sungjun과 Jumanji를 내적하는 코드
np.dot(sungjun_vector, Jumanji_vector)

# 쥬만지를 72만큼 좋아해~

0.7252029

In [42]:
# 그럼 내가 높게 준 다른 것도 높게 나오려나?

print("내가 5점 준 영화들 :",my_favorite_movies)
display(ratings.tail())
print("=======================================")
print("상위 30개 목록 :",sorted(movie_dict.items()))

내가 5점 준 영화들 : ['Alien (1979)', 'Raging Bull (1980)', 'Pretty Woman (1990)', 'Silence of the Lambs, The (1991)', 'Jumanji (1995)']


,user_id,movie_id,count,timestamp
0,6041,1196,5,978298730
1,6041,1210,5,978298730
2,6041,589,5,978298730
3,6041,593,5,978298730
4,6041,1,5,978298730


상위 30개 목록 : [(1, 'Jumanji (1995)'), (110, 'Rumble in the Bronx (1995)'), (260, 'Ladybird Ladybird (1994)'), (296, 'Priest (1994)'), (318, 'Strawberry and Chocolate (Fresa y chocolate) (1993)'), (356, 'I Love Trouble (1994)'), (457, 'Go Fish (1994)'), (480, 'Lassie (1994)'), (527, 'Secret Garden, The (1993)'), (589, 'Silence of the Lambs, The (1991)'), (593, 'Pretty Woman (1990)'), (608, 'Pallbearer, The (1996)'), (858, 'Kansas City (1996)'), (1097, 'Associate, The (1996)'), (1196, 'Alien (1979)'), (1197, 'Army of Darkness (1993)'), (1198, 'Big Blue, The (Le Grand Bleu) (1988)'), (1210, 'Raging Bull (1980)'), (1240, 'M (1931)'), (1265, 'Heathers (1989)'), (1270, 'Some Kind of Wonderful (1987)'), (1580, 'Wishmaster (1997)'), (1617, 'Stripes (1981)'), (2028, 'Something Wicked This Way Comes (1983)'), (2396, 'Deadly Friend (1986)'), (2571, 'Superman (1978)'), (2716, 'Tales of Terror (1962)'), (2762, 'Dog of Flanders, A (1999)'), (2858, 'Brief Encounter (1946)'), (2997, 'Tora! Tora! Tora! (

In [43]:
# # Alien에 대한 선호도 예측 --> Alien는 1196 입니당
alien = movie_to_idx[1196]
alien_vector = als_model.item_factors[alien]
np.dot(sungjun_vector, alien_vector)

# 결과는 구데기 of 구데기였다!!! --> 뭔가 문제가 있다!!


0.028930597

In [44]:
# 에이씨 인자를 잘못 넣었고만...
# print(sorted(movie_to_idx.items())) --> 내가 넣은 것은 쥬만지라는 movie_id가 아니라 1번 인덱스를 갖는 영화를 넣은 것

# 진짜 쥬만지를 찾아봅시다.
for idx, movie_id in movie_to_idx.items(): 
    if movie_id == 1:
        print("진짜진짜 쥬만지 :",idx)


# 진짜 쥬만지는 661 이었다!!!!
real_Jumanji = movie_to_idx[661]
real_Jumanji_vector = als_model.item_factors[real_Jumanji]
np.dot(sungjun_vector, real_Jumanji_vector)

# 진짜 쥬만지의 값도 72퍼엿다!!! --> 잘못나온 거 아닌가? 값이 완전히 똑같은데?

진짜진짜 쥬만지 : 661


0.7252029

In [46]:
# 아니 그러면 도대체 1번 인덱스 영화는 뭐길래...?

for idx, movie_id in movie_to_idx.items(): 
    if idx == 1:
        print("가짜가짜 쥬만지 :",movie_id)

print(movies['title'][1])
print(movies['title'][40])

# 박성준은 리차드 3세 라는 영화를 쥬만지만큼 좋아합니다.

가짜가짜 쥬만지 : 40
Jumanji (1995)
Richard III (1995)


In [49]:
# Alien으로 다시 재도전합니다.

for idx, movie_id in movie_to_idx.items(): 
    if movie_id == 1196:
        print("진짜진짜 에일리언 :",idx)
        
        
real_alien = movie_to_idx[185]
real_alien_vector = als_model.item_factors[real_alien]
np.dot(sungjun_vector, real_alien_vector)

# 좋아 정상이다.

진짜진짜 에일리언 : 185


0.7804109

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [53]:
# 내가 좋아하는 영화를 찾아 추천을 받아보자.
# 우선 내가 좋아하는 영화 아이디가 뭔지부터 알아보자.
# 난 양들의 침묵 (589)를 좋아한다.
# 근데 진짜진짜 양들의 침묵 위치를 알아야겠지?

for idx, movie_id in movie_to_idx.items(): 
    if movie_id == 589:
        print("진짜진짜 양들의 침묵 :",idx)

# 양들의 침묵은 918 이다.
favorite_movie = 918
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

진짜진짜 양들의 침묵 : 918


[(589, 1.0000002),
 (2571, 0.76741177),
 (480, 0.7253054),
 (2916, 0.69922984),
 (1580, 0.63301027),
 (1240, 0.5898938),
 (457, 0.51452804),
 (110, 0.51249325),
 (1610, 0.45030692),
 (989, 0.43664956),
 (1573, 0.3955438),
 (1200, 0.3834329),
 (1214, 0.38044757),
 (1527, 0.359547),
 (1544, 0.35772797)]

In [67]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

new_list = []
for i in similar_movie :
    new_list.append(idx_to_movie[i[0]])

print(new_list)

print(movies['title'][2276])
print(movies['title'][480]) # 아니 양들의 침묵은 범죄영화인데, 이렇게 아름답고 순수한 영화를 비슷한 영화라고 추천하다니..

[918, 2276, 1223, 2965, 371, 381, 282, 1265, 1981, 3930, 1014, 11, 2906, 976, 3807]
Desert Bloom (1986)
Lassie (1994)


### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [68]:
# 유저에게 아티스트 추천하기

user = user_to_idx[6039]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(2664, 0.9969192),
 (3551, 0.9157687),
 (1247, 0.8898051),
 (2300, 0.82871354),
 (1196, 0.82859695),
 (1084, 0.8104147),
 (1036, 0.79854673),
 (1104, 0.7975377),
 (1103, 0.79647225),
 (2871, 0.79142416),
 (2571, 0.7840972),
 (3527, 0.783605),
 (2917, 0.77246714),
 (1394, 0.76930624),
 (1748, 0.768474),
 (1206, 0.76294184),
 (1228, 0.74534833),
 (1041, 0.7382301),
 (1, 0.73464775),
 (2819, 0.73256063)]

In [70]:
recommended_list = []
for i in movie_recommended :
    recommended_list.append(idx_to_movie[i[0]])

print(recommended_list)
print(movies['title'][809]) # 애련의 장미라니 초월번역 지렸다..



[809, 572, 463, 2361, 185, 1222, 3000, 2726, 674, 3240, 2276, 679, 2516, 3599, 188, 1888, 361, 2122, 661, 3339]
Death in the Garden (Mort en ce jardin, La) (1956)


In [74]:
rose  = movie_to_idx[809]
explain = als_model.explain(user, csr_data, itemid=809)
for i in explain[1] :
    print(idx_to_movie[i[0]])
    
print(movies['title'][707]) # 너무 고전이라 감도 안온다.

276
707
1739
1285
1199
947
212
2342
3184
305
Switchblade Sisters (1975)
